<a href="https://colab.research.google.com/github/Pasha302/dataset_titinic/blob/master/horse_or_human.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip


!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

--2020-06-24 18:52:16--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 149574867 (143M) [application/zip]
Saving to: ‘/tmp/horse-or-human.zip’

/tmp/horse-or-human 100%[===================>] 142.65M  86.2MB/s    in 1.7s    

2020-06-24 18:52:18 (86.2 MB/s) - ‘/tmp/horse-or-human.zip’ saved [149574867/149574867]

--2020-06-24 18:52:19--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11480187 (11M) [application/zip]

In [3]:
import os 
import zipfile
local_zip ='/tmp/horse-or-human.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp/horse-or-human')
local_zip='/tmp/validation-horse-or-human.zip'
zip_ref=zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp/validation-horse-or-human')
zip_ref.close()

In [4]:
train_horse_dir = os.path.join('/tmp/horse-or-human/horses')
train_human_dir = os.path.join('/tmp/horse-or-human/humans')
validation_horse_dir = os.path.join('/tmp/validation-horse-or-human/horses')
validation_human_dir = os.path.join('/tmp/validation-horse-or-human/humans')

train_horse_names=os.listdir(train_horse_dir)
train_human_names=os.listdir( train_human_dir)

validation_horse_names=os.listdir(validation_horse_dir)

validation_human_names=os.listdir(validation_human_dir)




In [5]:
import tensorflow as tf


In [6]:
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(150,150,3)),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPool2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation='relu'),
                                    tf.keras.layers.Dense(1,activation='sigmoid')
])




In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [8]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',optimizer=RMSprop(lr=0.001),metrics=['accuracy'])


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen=ImageDataGenerator(rescale=1/255)
validation_datagen=ImageDataGenerator(rescale=1/255)

train_generator =train_datagen.flow_from_directory(
    '/tmp/horse-or-human/',
    target_size=(150,150),
    batch_size=128,
    class_mode='binary'
)

validation_generator= validation_datagen.flow_from_directory(
    '/tmp/validation-horse-or-human/',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
    
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [10]:
history =model.fit(
    train_generator,
    steps_per_epoch=8,
    epochs=15,
    verbose=1,
    validation_data=validation_generator,
    validation_steps=8
  
)

Epoch 1/15
8/8 [==============================] - 5s 659ms/step - loss: 3.1992 - accuracy: 0.5495 - val_loss: 0.6354 - val_accuracy: 0.5000
Epoch 2/15
8/8 [==============================] - 6s 709ms/step - loss: 0.6288 - accuracy: 0.5695 - val_loss: 0.4157 - val_accuracy: 0.8086
Epoch 3/15
8/8 [==============================] - 6s 707ms/step - loss: 0.3216 - accuracy: 0.8932 - val_loss: 1.4129 - val_accuracy: 0.7109
Epoch 4/15
8/8 [==============================] - 6s 709ms/step - loss: 0.7404 - accuracy: 0.7531 - val_loss: 2.5114 - val_accuracy: 0.5352
Epoch 5/15
8/8 [==============================] - 6s 800ms/step - loss: 0.3088 - accuracy: 0.9010 - val_loss: 1.0541 - val_accuracy: 0.7070
Epoch 6/15
8/8 [==============================] - 6s 699ms/step - loss: 0.1119 - accuracy: 0.9600 - val_loss: 1.1912 - val_accuracy: 0.8125
Epoch 7/15
8/8 [==============================] - 6s 748ms/step - loss: 0.1010 - accuracy: 0.9639 - val_loss: 0.3134 - val_accuracy: 0.9062
Epoch 8/15
8/8 [====

In [30]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded =files.upload()
for fn in uploaded.keys():
  path='/content/' + fn
  img = image.load_img(path,target_size=(150,150))
  '''
  print(type(img))
  x =image.img_to_array(img)
  print(x.shape)
  x= np.expand_dims(x,axis=0)
  print(x.shape)
  
  images = np.vstack([x])
  print(x.shape)
  '''
  classes =model.predict(images,batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn+ "is a human")
  else: print(fn+ "is a horse")

Saving Runner_topNteaser.jpg to Runner_topNteaser.jpg
[1.]
Runner_topNteaser.jpgis a human


In [25]:
x=np.ones((3,3,2))
print(type([x]))


<class 'list'>
